In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, sys
sys.path.append('../Shared')
import processing, Model, Visualization, modules

def get_label_from_filename(fname):
    for cls, label in class_to_label.items():
        if cls in fname:
            return label
    return None  # if no class found


Label 파일에서 Status랑 Group있는데 걸음걸이 마다 랑 뭐 다르게 준것같음 - 그냥 무시하고 전체 레이블로 가기

In [30]:
classes = ['STC', 'WAK', 'STDUP', 'SITDN', 'UPS', 'DNS', 'KLFT', 'TPTO', 'LLF', 'LLB', 'LLS', 'KLCL', 'HS', 'TO', 'LUGF', 'LUGB'] #16 classes

Features = ['mav', 'var', 'zc', 'iemg', 'wl', 'wamp', 'mavs', 'rms', 'ssc', 'msq', 'v3', 'ld', 'dabs', 'mfl', 'mpr', 'mnf', 'psr', 'arc1', 'arc2', 'arc3', 'arc4', 'cc1', 'cc2', 'cc3', 'cca', 'dwtc1', 'dwtc2', 'dwtpc1', 'dwtpc2', 'dwtpc3']

base_path = 'D:/Data/Gait-EMG/SIAT/SIAT_LLMD20230404/'
Sub_lst_data = [f"Sub{i:02d}"+"/Data/" for i in range(1, 41)]
Sub_lst_label = [f"Sub{i:02d}"+"/Labels/" for i in range(1, 41)]

In [38]:
###나중에 여기부터 for (Subject 단위) - 지금부터 1 subject 시작
data_lst = os.listdir(base_path + Sub_lst_data[0])
label_lst = os.listdir(base_path + Sub_lst_label[0])
#label = pd.read_csv(base_path + Sub_lst_label[0] + label_lst[0])

class_to_label = {cls: i for i, cls in enumerate(classes)}

data_lst = os.listdir(base_path + Sub_lst_data[0])
label_lst = os.listdir(base_path + Sub_lst_label[0])

dataset, labels = [], []
for l, d in zip(label_lst, data_lst):
    print(l, d)
    data = pd.read_csv(base_path + Sub_lst_data[0] + "/" + d)
    data = data.iloc[:, 1:]   # 첫 column(time) 제거

    label = get_label_from_filename(l)

    labels.append(np.array([label] * data.shape[0]))
    dataset.append(data)
    print(np.array([label] * data.shape[0]).shape, data.shape, "\n")

Sub01_DNS_Label.csv Sub01_DNS_Data.csv
(30668,) (30668, 25) 

Sub01_HS_Label.csv Sub01_HS_Data.csv
(94817,) (94817, 25) 

Sub01_KLCL_Label.csv Sub01_KLCL_Data.csv
(101249,) (101249, 25) 

Sub01_KLFT_Label.csv Sub01_KLFT_Data.csv
(104193,) (104193, 25) 

Sub01_LLB_Label.csv Sub01_LLB_Data.csv
(91745,) (91745, 25) 

Sub01_LLF_Label.csv Sub01_LLF_Data.csv
(92417,) (92417, 25) 

Sub01_LLS_Label.csv Sub01_LLS_Data.csv
(103041,) (103041, 25) 

Sub01_LUGB_Label.csv Sub01_LUGB_Data.csv
(101281,) (101281, 25) 

Sub01_LUGF_Label.csv Sub01_LUGF_Data.csv
(98913,) (98913, 25) 

Sub01_SITDN_Label.csv Sub01_SITDN_Data.csv
(84645,) (84645, 25) 

Sub01_STC_Label.csv Sub01_STC_Data.csv
(47845,) (47845, 25) 

Sub01_STDUP_Label.csv Sub01_STDUP_Data.csv
(81925,) (81925, 25) 

Sub01_TO_Label.csv Sub01_TO_Data.csv
(93633,) (93633, 25) 

Sub01_TPTO_Label.csv Sub01_TPTO_Data.csv
(101313,) (101313, 25) 

Sub01_UPS_Label.csv Sub01_UPS_Data.csv
(35479,) (35479, 25) 

Sub01_WAK_Label.csv Sub01_WAK_Data.csv
(27164,

In [41]:
#pd.Series(labels[0]).value_counts()

D = np.concatenate(dataset, axis=0)
L = np.concatenate(labels, axis=0)

print(D.shape, L.shape)

(1290328, 25) (1290328,)


In [45]:
balanced_idx = modules.Downsample_to_balance_class(L)
X_emg_bal, y_bal = D[balanced_idx], L[balanced_idx]
print("EMG:", X_emg_bal.shape, "y:", y_bal.shape)

X_emg_bal, y_bal = modules.random_downsample(X_emg_bal, y_bal, fraction=0.5)
print("After downsampling - EMG:", X_emg_bal.shape, "y:", y_bal.shape)

EMG: (434624, 25) y: (434624,)
After downsampling - EMG: (217312, 25) y: (217312,)


In [46]:
window_size, step_size = 200, 10
num_channels, num_features = X_emg_bal.shape[1], 30

X_tmp, y_tmp = processing.sliding_window_with_labels(X=X_emg_bal, y=y_bal,  window_size=window_size, step_size=step_size)  # (num_windows, win_len, ch)

all_X, all_y = modules.get_X_y(X_tmp, y_tmp)
all_y = modules.y_change_to_int(all_y)
all_X = all_X.reshape(-1, num_channels, num_features, 1)

C:\Users\hml76\PycharmProjects\Cross-motor-decoding\github\Lower-limb\Gait3-SAIT\../Shared\processing.py:85: RuntimeWarning: divide by zero encountered in log
  spectrum = np.log(np.abs(fft(signal)))


Unique labels: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
New labels: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]


In [ ]:
from sklearn.model_selection import train_test_split

Sub_acc = []
for feature_idx in range(num_features):
    X_feature = all_X[:, :, feature_idx]   # shape: (samples, num_channels, 1, 1)
    X_feature = X_feature.reshape(-1, num_channels, 1)
    X_train, X_test, y_train, y_test = train_test_split(X_feature, all_y, test_size=0.2, random_state=42)

    model = Model.build_model_1D(input_shape=X_feature.shape[1:], num_classes=len(np.unique(all_y)))
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=512, verbose=0)

    print(f"\t Feature {feature_idx} ({Features[feature_idx]}): "
        f"val acc => {np.max(history.history['val_accuracy'])*100:.2f}%")
    Sub_acc.append(np.max(history.history['val_accuracy'])*100)

C:\Users\hml76\PycharmProjects\Cross-motor-decoding\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
